# Main
Orchestrates the full pipeline with `%run` notebook modules.


In [28]:
%run ../tools/tools.ipynb


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [29]:
%run ../data/data_pipeline.ipynb


In [30]:
%run ../modeling/modeling_pipeline.ipynb


In [31]:
compustat_csv_path = "../data/compustat.csv"
price_cache_path = "../data/adj_close_cache.pkl"
fixed_end_date = "2025-12-31"
force_refresh = False

data_outputs = run_data_pipeline(
    compustat_csv_path=compustat_csv_path,
    fixed_end_date=fixed_end_date,
    price_cache_path=price_cache_path,
    force_refresh=force_refresh,
    download_threads=False,
)
modeling_panel = data_outputs["modeling_panel"]

modeling_panel.head()


Compustat tickers: 100
Price window: 2009-11-27 to 2025-12-31
Loaded cached prices from ../data/adj_close_cache.pkl
Downloaded/loaded adjusted-close prices for 100 tickers
Panel rows (pre-fundamentals): 18970
Panel rows (final): 16337
Date range: 2012-01-31 to 2025-11-30


,month_end,ticker,adj_close,vol_60,momentum_6_12,target_return,availability_date,revenue_growth,roe
0,2012-01-31,AAPL,13.690310,0.015748,0.150772,0.188311,2012-01-01,0.659624,0.338341
1,2012-02-29,AAPL,16.268341,0.013484,0.089522,0.105283,2012-01-01,0.659624,0.338341
2,2012-03-31,AAPL,17.981123,0.014413,0.094143,-0.025969,2012-01-01,0.659624,0.338341
3,2012-04-30,AAPL,17.514166,0.020699,0.156085,-0.010702,2012-01-01,0.659624,0.338341
4,2012-05-31,AAPL,17.326725,0.022435,0.098813,0.010853,2012-01-01,0.659624,0.338341


In [32]:
summarize_modeling_panel(modeling_panel)


Rows in final panel: 16337
Unique months: 167
Missing values by column:
month_end            0
ticker               0
adj_close            0
vol_60               0
momentum_6_12        0
target_return        0
availability_date    0
revenue_growth       0
roe                  0
dtype: int64
month_end
2012-01-31    93
2012-02-29    93
2012-03-31    93
2012-04-30    93
2012-05-31    93
              ..
2021-04-30    99
2021-05-31    99
2021-06-30    99
2021-07-31    99
2021-08-31    99
Name: count, Length: 116, dtype: int64


,month_end,adj_close,vol_60,momentum_6_12,target_return,availability_date,revenue_growth,roe
count,16337,16337.000000,16337.000000,16337.000000,16337.000000,16337,16337.000000,16337.000000
mean,2019-01-29 02:34:57.374059008,130.078425,0.016174,0.087267,0.014754,2018-07-17 19:14:35.485095168,0.085621,0.342528
min,2012-01-31 00:00:00,0.274592,0.003945,-0.761706,-0.554274,2012-01-01 00:00:00,-0.748907,-33.733333
25%,2015-08-31 00:00:00,39.998188,0.010898,-0.027048,-0.028644,2015-01-01 00:00:00,0.000943,0.093993
50%,2019-01-31 00:00:00,70.534248,0.014137,0.073063,0.013522,2019-01-01 00:00:00,0.055275,0.173320
75%,2022-07-31 00:00:00,138.218781,0.018636,0.178829,0.055479,2022-01-01 00:00:00,0.137492,0.300221
max,2025-11-30 00:00:00,5768.254395,0.092889,4.103457,0.898065,2025-01-01 00:00:00,3.872273,70.384615
std,NaN,257.234818,0.008344,0.212237,0.077210,NaN,0.206447,2.879983


In [33]:
model_outputs = run_modeling_pipeline(modeling_panel)


Train rows: 12937 | Test rows: 3400
Split month: 2023-01-31
Table 1. Flip Summary by Threshold


,threshold,flip_rate,flips_total,linear_approve_rf_reject,rf_approve_linear_reject
0,0.025,0.2406,818,113,705
1,0.050,0.0585,199,3,196
2,0.075,0.0194,66,0,66
3,0.100,0.0071,24,0,24


Table 2. Decision Counts and Error Counts (Side-by-Side)


approved_count                rejected_count               \
model     LinearRegression RandomForest LinearRegression RandomForest   
threshold                                                               
0.025                  296          888             3104         2512   
0.050                   13          206             3387         3194   
0.075                    0           66             3400         3334   
0.100                    0           24             3400         3376   

            type_II_errors                 type_I_errors               
model     LinearRegression RandomForest LinearRegression RandomForest  
threshold                                                              
0.025                 1752         1423              105          368  
0.050                 1938         1815                8           78  
0.075                 1943         1902                0           25  
0.100                 1943         1930                0           11

Table 3. Economic Impact (Realized Return Sums, Side-by-Side)


approved_return_sum              rejected_return_sum             
model        LinearRegression RandomForest    LinearRegression RandomForest
threshold                                                                  
0.025               14.217821    21.417989           44.496844    37.296675
0.050                0.404447     8.477325           58.310218    50.237339
0.075                0.000000     3.238277           58.714664    55.476387
0.100                0.000000     1.020189           58.714664    57.694475

Table 4. Average Realized Returns (Side-by-Side)


approved_return_avg              rejected_return_avg             
model        LinearRegression RandomForest    LinearRegression RandomForest
threshold                                                                  
0.025                0.048033     0.024119            0.014335     0.014847
0.050                0.031111     0.041152            0.017216     0.015729
0.075                     NaN     0.049065            0.017269     0.016640
0.100                     NaN     0.042508            0.017269     0.017090

Detailed Long-Format Results


,model,threshold,approved_count,rejected_count,type_I_errors,type_II_errors,approved_return_sum,rejected_return_sum,approved_return_avg,rejected_return_avg
0,LinearRegression,0.025,296,3104,105,1752,14.217821,44.496844,0.048033,0.014335
1,RandomForest,0.025,888,2512,368,1423,21.417989,37.296675,0.024119,0.014847
2,LinearRegression,0.050,13,3387,8,1938,0.404447,58.310218,0.031111,0.017216
3,RandomForest,0.050,206,3194,78,1815,8.477325,50.237339,0.041152,0.015729
4,LinearRegression,0.075,0,3400,0,1943,0.000000,58.714664,NaN,0.017269
5,RandomForest,0.075,66,3334,25,1902,3.238277,55.476387,0.049065,0.016640
6,LinearRegression,0.100,0,3400,0,1943,0.000000,58.714664,NaN,0.017269
7,RandomForest,0.100,24,3376,11,1930,1.020189,57.694475,0.042508,0.017090
